# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow

tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-macos==2.10.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXWY2IH/imdb_reviews-train.tfrecord*...…

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXWY2IH/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXWY2IH/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2023-02-24 14:52:58.854233: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [38]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

In [6]:
wv['dog']

array([-1.31179795e-01,  1.29680246e-01, -1.01218492e-01,  2.27671757e-01,
       -5.70501834e-02, -3.18735570e-01, -4.04647057e-04,  5.27364075e-01,
       -1.48557544e-01, -1.26766711e-01, -2.01488305e-02, -2.69839048e-01,
       -3.92134935e-02,  1.16032600e-01, -3.78698967e-02, -2.06698269e-01,
        1.62418962e-01, -2.72651941e-01, -5.43630198e-02, -3.70347530e-01,
        1.59486964e-01,  1.00631885e-01,  2.49377921e-01, -1.07569791e-01,
        3.92711163e-03,  4.49785106e-02, -1.94976047e-01, -1.01393051e-01,
       -2.12458923e-01,  2.35036090e-02,  1.63787693e-01,  2.78089084e-02,
        1.60475284e-01, -3.73258561e-01, -1.82365865e-01,  2.41694003e-01,
        1.07034318e-01, -1.27999663e-01, -2.94709295e-01, -3.26329887e-01,
       -1.50400206e-01, -2.42325261e-01, -2.75756836e-01,  2.23004058e-01,
        2.66381890e-01, -7.25778416e-02, -9.93703753e-02, -3.07150241e-02,
        2.19239220e-01,  1.49099782e-01,  6.66708574e-02, -1.61591947e-01,
       -2.48242676e-01, -

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [17]:
wv['chair']

array([-0.08840887,  0.05286748, -0.06087678,  0.11259277,  0.01181867,
       -0.146135  ,  0.07997188,  0.27106544, -0.12103471, -0.11774917,
       -0.05720159, -0.17411551,  0.01537064,  0.03728839, -0.03027301,
       -0.10005415,  0.0999897 , -0.0896476 ,  0.0149204 , -0.19174387,
        0.08456607,  0.0416607 ,  0.12676288, -0.09677342, -0.03082684,
        0.01121828, -0.13680917, -0.03520885, -0.12012321, -0.01797676,
        0.1343972 ,  0.0081268 ,  0.07579023, -0.21419817, -0.17384087,
        0.16534624,  0.05926875, -0.10001062, -0.17515208, -0.17214024,
       -0.04934389, -0.12110963, -0.12012482,  0.06384189,  0.17171039,
       -0.03050609, -0.03326181, -0.07949048,  0.15600973,  0.11245935,
        0.0866762 , -0.11370517, -0.10698526,  0.0017948 , -0.10322852,
        0.09805708,  0.10919227, -0.0183614 , -0.06905659,  0.00938475,
        0.06342769, -0.01460001, -0.01203527,  0.01476273, -0.0524267 ,
        0.10239514, -0.03465737,  0.03869641, -0.12496661,  0.09

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [20]:
wv['chair'].size

100

In [23]:
wv['dog'].shape

(100,)

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

In [33]:
word2vec.wv.most_similar('fishing', topn=10)

[('educated', 0.9814343452453613),
 ('drinking', 0.9809631705284119),
 ('gang', 0.9806191325187683),
 ('princess', 0.9798257946968079),
 ('angeles', 0.979061484336853),
 ('enter', 0.9786893129348755),
 ('field', 0.9786695837974548),
 ('gold', 0.9782401919364929),
 ('manager', 0.9776360392570496),
 ('fu', 0.9769474267959595)]

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

[('man', 0.981688916683197),
 ('woman', 0.9367076754570007),
 ('girl', 0.9329282641410828),
 ('boy', 0.8800891041755676),
 ('guy', 0.8668673634529114),
 ('child', 0.8077195286750793),
 ('role', 0.8070240020751953),
 ('lady', 0.8013709187507629),
 ('husband', 0.7949036359786987),
 ('wife', 0.7874804139137268)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [39]:
v_queen = word2vec.wv['queen']
v_king = word2vec.wv['king']
v_man = word2vec.wv['man']

v_result = v_queen - v_king + v_man

word2vec.wv.similar_by_vector(v_result)


[('man', 0.9856676459312439),
 ('woman', 0.9339273571968079),
 ('girl', 0.9141790866851807),
 ('wife', 0.886448860168457),
 ('young', 0.8594347238540649),
 ('father', 0.8547314405441284),
 ('role', 0.8541600704193115),
 ('guy', 0.8531272411346436),
 ('boy', 0.8449637293815613),
 ('husband', 0.8405648469924927)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [43]:
v_good = word2vec.wv['good']
v_bad = word2vec.wv['bad']
v_result = v_good - v_bad
word2vec.wv.similar_by_vector(v_result)

[('who', 0.4020281732082367),
 ('by', 0.3709651529788971),
 ('young', 0.318206787109375),
 ('man', 0.26937979459762573),
 ('played', 0.2559031844139099),
 ('his', 0.2433338612318039),
 ('been', 0.23589196801185608),
 ('a', 0.23189525306224823),
 ('sibling', 0.23157699406147003),
 ('woman', 0.22328327596187592)]

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [45]:
v_stupid = word2vec.wv['stupid']
res = v_good - v_bad + v_stupid

[('nice', 0.7630822062492371),
 ('good', 0.7413495779037476),
 ('great', 0.7354967594146729),
 ('also', 0.7187266945838928),
 ('decent', 0.7119225263595581),
 ('done', 0.7014288902282715),
 ('always', 0.6950615644454956),
 ('baldwin', 0.694158136844635),
 ('biased', 0.6939597725868225),
 ('potential', 0.6939337849617004)]

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [48]:
word2vec.wv.similar_by_vector(res)

[('nice', 0.7630822062492371),
 ('good', 0.7413495779037476),
 ('great', 0.7354967594146729),
 ('also', 0.7187266945838928),
 ('decent', 0.7119225263595581),
 ('done', 0.7014288902282715),
 ('always', 0.6950615644454956),
 ('baldwin', 0.694158136844635),
 ('biased', 0.6939597725868225),
 ('potential', 0.6939337849617004)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [63]:
v_boy = word2vec.wv['boy']
v_girl = word2vec.wv['girl']
v_man = word2vec.wv['man']
v_woman = word2vec.wv['woman']

res1 = v_girl + v_man -v_woman
res2 =  v_boy
word2vec.wv.similar_by_vector(res1)

[('man', 0.9804842472076416),
 ('girl', 0.9653876423835754),
 ('woman', 0.9410542249679565),
 ('young', 0.8867107629776001),
 ('wife', 0.8832592368125916),
 ('boy', 0.8730332851409912),
 ('father', 0.8721991777420044),
 ('guy', 0.8697517514228821),
 ('mother', 0.8593718409538269),
 ('himself', 0.8567374348640442)]

In [64]:
word2vec.wv.similar_by_vector(res2)

[('boy', 1.0000001192092896),
 ('married', 0.9687053561210632),
 ('conscience', 0.9608739614486694),
 ('lady', 0.9604454636573792),
 ('daughter', 0.9591637253761292),
 ('son', 0.9590871930122375),
 ('brother', 0.9575938582420349),
 ('woman', 0.9572828412055969),
 ('herself', 0.9551699757575989),
 ('struggles', 0.9551449418067932)]

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [65]:
word2vec2 = Word2Vec(sentences=X_train, vector_size=100)

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [69]:
word2vec2.wv.key_to_index

{'the': 0,
 'a': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'is': 5,
 'br': 6,
 'in': 7,
 'i': 8,
 'it': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'but': 16,
 'movie': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'his': 22,
 'are': 23,
 'have': 24,
 'one': 25,
 'be': 26,
 'he': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'they': 31,
 'an': 32,
 'so': 33,
 'like': 34,
 'who': 35,
 'from': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'if': 40,
 'out': 41,
 'about': 42,
 "it's": 43,
 'has': 44,
 'what': 45,
 'some': 46,
 'there': 47,
 'good': 48,
 'more': 49,
 'when': 50,
 'very': 51,
 'no': 52,
 'up': 53,
 'she': 54,
 'my': 55,
 'time': 56,
 'even': 57,
 'which': 58,
 'would': 59,
 'really': 60,
 'only': 61,
 'had': 62,
 'story': 63,
 'me': 64,
 'see': 65,
 'can': 66,
 'their': 67,
 'were': 68,
 'well': 69,
 'than': 70,
 'much': 71,
 'get': 72,
 'do': 73,
 'great': 74,
 'been': 75,
 'we': 76,
 'first': 77,
 'bad': 78,
 'because': 79,
 'into': 80,
 'other': 81,


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [67]:
word2vec3 = Word2Vec(sentences=X_train, vector_size=100, min_count=10)
word2vec4 = Word2Vec(sentences=X_train, vector_size=100, min_count=3)

Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [70]:
word2vec3 = Word2Vec(sentences=X_train, vector_size=100, min_count=10, window=10)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [74]:
emedding = np.array()

TypeError: array() missing required argument 'object' (pos 0)

In [ ]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embedding = []
    for word in sentence:
        if word in word2vec.wv:
            embedding.append(word2vec.wv[word])
    return np.array(embedding)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

In [76]:
embed_sentence(word2vec,example)

array([[ 6.00567460e-01, -9.77364421e-01,  1.22312117e+00,
        -2.89608300e-01, -2.83019513e-01, -6.44604564e-01,
        -5.58074951e-01, -4.98849779e-01, -7.10611284e-01,
        -6.44478261e-01,  8.70161235e-01,  2.68628955e-01,
         2.64924429e-02, -1.27949566e-01, -4.98828828e-01,
         3.37203383e-01,  8.66689980e-01, -1.42999291e+00,
        -4.45500016e-01, -2.03647804e+00, -4.52093780e-01,
        -7.37015903e-02,  1.12146533e+00,  5.24856150e-01,
        -1.69498295e-01, -1.03027068e-01, -1.02931607e+00,
        -1.66133240e-01,  1.37516245e-01, -6.59305751e-01,
         1.19658418e-01,  1.58884346e-01,  1.78241932e+00,
         1.46520317e+00,  2.08722845e-01,  1.76379848e+00,
         7.47298002e-02,  2.02249080e-01, -7.75541782e-01,
        -5.25325909e-02, -1.99741378e-01,  2.88333863e-01,
        -7.37056673e-01,  9.73172188e-02, -6.17136478e-01,
         2.42210433e-01, -1.79511264e-01, -3.40417981e-01,
        -2.41466999e-01,  3.73243421e-01, -1.29998159e+0

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [ ]:
def embedding(word2vec, sentences):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec,sentence)
        embedded_sentences.append(embedded_sentence)
    return embedded_sentences
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', value=0)
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', value=0)
assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)

In [83]:
!git add .
!git commit -m"understand embeddin



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
